# Downloads and Imports

In [11]:
import nltk
import pandas as pd
import urllib.request
from difflib import ndiff
from collections import Counter
from nltk.corpus import wordnet as wn
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [37]:
import sys
sys.path.append('../')

from utils.tools import DistanceTools
from utils.setup import SetupFunctions
from utils.evaluation import Evaluation

In [38]:
tools = DistanceTools()
setup = SetupFunctions()
evals = Evaluation()

In [25]:
from tools import *

ModuleNotFoundError: No module named 'DistanceTools'

In [136]:
help(st)

Help on module utils.setup in utils:

NAME
    utils.setup

FUNCTIONS
    buildDictionary()

    countWords(lst)

    describeData(df, repeats, word_count)

DATA
    __warningregistry__ = {'version': 59}
    wn = <WordNetCorpusReader in '.../corpora/wordnet' (not loaded yet)>

FILE
    c:\users\lenovo\documents\repos\spell-correction-using-med\utils\setup.py




In [132]:
st.countWords

<function utils.setup.countWords(lst)>

In [1]:
from utils.tools import *
from utils.setup import findPossibleFirstLetters
from utils.evaluation import *

ModuleNotFoundError: No module named 'utils'

In [7]:
def createData(link, filename = "../data//data.dat"):

  urllib.request.urlretrieve(link, filename)
  current_key = None
  result_dict = {}
  repeats = []
  print('About the spellcheck dataset:\n')
  for line in open(filename).readlines():

      lines = line.strip().split('\n')

      if lines[0].startswith('$'):
              current_key = lines[0][1:].lower()
              if(current_key not in result_dict.keys()):
                  result_dict[current_key] = []
              else:
                print('The word "' + current_key + '" occurs more than once')
                repeats.append(current_key)
                #result_dict[current_key].append(lines[0].lower())
              #print(line[1:-2])
              #result_dict[current_key] = []
      else:
              result_dict[current_key].append(lines[0].lower())

  df = pd.DataFrame([result_dict.keys(), result_dict.values()]).T
  df.columns = ['words', 'misspellings']

  return df, repeats

# For parallel processing

In [8]:
from pandarallel import pandarallel
pandarallel.initialize(nb_workers = 300)

INFO: Pandarallel will run on 300 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [9]:
import json

In [33]:
def findingClosestWords(start_index = 0, end_index = 10, df =None, 
                        k_vals = [1,5,10], 
                        link =  "https://www.dcs.bbk.ac.uk/~roger/missp.dat"):

  from utils.tools import DistanceTools as tools
  
  repeats = []
  if (df is None):
    df, repeats = createData(link = link)  #can mention filename and link here

  df, starters = setup.findPossibleFirstLetters(df)
  dictionary, word_count = setup.buildDictionary()
  setup.describeData(df, repeats, word_count)
  df1 = df[start_index : end_index]
  df1['k_list'] = df1.parallel_apply(lambda x: tools.find_k(x.words, x.misspellings, dictionary, starters), axis = 1)
  results = evals.getResults(df)
  print('\n')
  success_at_k = evals.getAverageSuccessValues(results, df.num_of_words.sum(), k_vals)
  df1.drop(columns = ['num_of_words'], inplace = True)
  df1.to_json('results\\birkbeckResults\\full\\spellCheck.json')
  with open("results\\birkbeckResults\\full\\evalResults.json", "w") as fp:
    json.dump(success_at_k, fp)
  print('-----------------------------------------------------------------------------------------------------')
  return df1

In [40]:
links = { 'birkbeck' : 'https://www.dcs.bbk.ac.uk/~roger/missp.dat',
        'aspell' : 'https://www.dcs.bbk.ac.uk/~roger/aspell.dat',
        'wikipedia' : 'https://www.dcs.bbk.ac.uk/~roger/wikipedia.dat' }
k_values = findingClosestWords(start_index = 1, end_index = 10, k_vals = [1,5,10], link = links['birkbeck'])

About the spellcheck dataset:

The word "cambridge" occurs more than once
The word "february" occurs more than once
The word "miss" occurs more than once
The word "monday" occurs more than once
The word "sunday" occurs more than once
The word "wednesday" occurs more than once


TypeError: SetupFunctions.findStarters() takes 3 positional arguments but 4 were given

In [38]:
import os
os.listdir('./examples')

#os.listdir('./results/birkbeckResults/parts')

FileNotFoundError: [WinError 3] The system cannot find the path specified: './examples'

In [56]:
arr = next(os.walk('../utils/'))[2]
print(arr)

['evaluation.py', 'init.py', 'setup.py', 'tools.py']


# Concatenate result files

In [218]:
def concatenateParallelRuns(k_vals = [1, 5, 10], num_of_words = 36133):
  df1 = pd.read_json('/content/bb_0_to_1000.json')
  df2 = pd.read_json('/content/bb_1000_to_2000.json')
  df3 = pd.read_json('/content/bb_2000_to_3000.json')
  df4 = pd.read_json('/content/bb_3000_to_4000.json')
  df5 = pd.read_json('/content/bb_4000_to_5000.json')
  df6 = pd.read_json('/content/bb_5000_to_6130.json')
  df = pd.concat([df1, df2, df3, df4,df5, df6])
  results = getResults(df)
  print('\n')
  success_at_k = getAverageSuccessValues(results, num_of_words, k_vals)
  df.to_json('results\birkbeckResults\full\spellCheck.json')
  with open("results\birkbeckResults\full\evalResults.json", "w") as fp:
    json.dump(success_at_k, fp)
  print('-----------------------------------------------------------------------------------------------------')
  return df

In [219]:
concatenateParallelRuns()

k =  4 in 2501 search(es)
k =  1 in 19162 search(es)
k =  2 in 7151 search(es)
k =  5 in 1376 search(es)
k =  3 in 4184 search(es)
k =  7 in 418 search(es)
k =  6 in 756 search(es)
k =  10 in 78 search(es)
k =  8 in 245 search(es)
k =  9 in 149 search(es)
k =  11 in 48 search(es)
k =  12 in 33 search(es)
k =  13 in 19 search(es)
k =  16 in 4 search(es)
k =  15 in 1 search(es)
k =  14 in 7 search(es)
k =  17 in 1 search(es)


Success at 1 is 53.03 %
Success at 5 is 95.13 %
Success at 10 is 99.69 %
-----------------------------------------------------------------------------------------------------


,words,misspellings,k_list
0,albert,[ab],[4]
1,america,"[ameraca, amercia]","[1, 1]"
2,american,[ameracan],[1]
3,april,[apirl],[1]
4,austrian,[austrain],[1]
...,...,...,...
6125,february,"[febuary, feburary, feburay, febuary]","[1, 1, 1, 1]"
6126,miss,"[mis, mess, mis, mrs]","[1, 1, 1, 2]"
6127,monday,"[munday, mond]","[1, 2]"
6128,sunday,"[sanday, sonday, sunbay, sundays, suntday]","[1, 1, 1, 1, 1]"
